In [88]:
class board:
    def __init__(self, txtfile):
        self.sqDim = 0
        self.ptBoard = []
        with open(txtfile, 'r') as f:
            while True:
                file_line = f.readline()
                if not file_line: 
                    break
                else:
                    self.sqDim += 1
                    row = file_line.replace("\r\n", '').split("\t")
                    for element in row:
                        self.ptBoard.append(int(element)) 
        self.size = self.sqDim**2
        self.maxPlayerTaken = []
        self.minPlayerTaken = []
        self.openPositions = [i for i in range(self.size)]
        
    def printBoard(self):
        for i in range(self.size):
            if i % 6 == 0:
                print ' '
            if i in self.maxPlayerTaken:
                print '[A]',
            elif i in self.minPlayerTaken:
                print '[B]',
            else:
                print '[ ]',
        print ' '

In [98]:
class warGame:
    def __init__(self, board):
        self.board = board
        self.maxScore = 0
        self.minScore = 0
        
    def utility(self):
        maxScore = 0
        minScore = 0
        for pos in self.board.maxPlayerTaken:
            maxScore += self.board.ptBoard[pos]
        for pos in self.board.minPlayerTaken:
            minScore += self.board.ptBoard[pos]
        return maxScore - minScore
            
    def calculateScore(self):
        self.maxScore = 0
        self.minScore = 0
        for pos in self.board.maxPlayerTaken:
            self.maxScore += self.board.ptBoard[pos]
        for pos in self.board.minPlayerTaken:
            self.minScore += self.board.ptBoard[pos]
    
    def isComplete(self):
        #if there are no open spaces
        if not self.board.openPositions:
            return True
        else:
            return False
        
    def runGame(self):
        #start with initial state
        player = 'max'
    
        while not self.isComplete():
            #max makes move
            if player == 'max':
                maxSeen = None
                move = None
                for openSpace in self.board.openPositions:
                    utility = self.minimax(openSpace, 'min', 1)
                    if maxSeen == None:
                        maxSeen = utility
                        move = openSpace
                    elif utility > maxSeen:
                        maxSeen = utility
                        move = openSpace
                self.makeMove(move, player)
                player = 'min'
        
            #min makes move
            else:
                minSeen = None
                move = None
                for openSpace in self.board.openPositions:
                    utility = self.minimax(openSpace, 'max', 1)
                    if minSeen == None:
                        minSeen = utility
                        move = openSpace
                    elif utility < minSeen:
                        minSeen = utility
                        move = openSpace
                self.makeMove(move, player)
                player = 'max'
                
        self.calculateScore()

    def minimax(self, pos, player, depth):
        maxDepth = 3
        conqueredPositions = self.makeMove(pos, player)
        if depth >= maxDepth or self.isComplete():
            self.reverseMove(pos, player, conqueredPositions)
            return self.utility()
        
        if player == 'max':
            maxSeen = None
            for openSpace in self.board.openPositions:
                utility = self.minimax(openSpace, 'min', depth + 1)
                if maxSeen == None:
                    maxSeen = utility
                    move = openSpace
                elif utility > maxSeen:
                    maxSeen = utility
                    move = openSpace
            self.reverseMove(pos, player, conqueredPositions)
            return utility
        else:
            minSeen = None
            for openSpace in self.board.openPositions:
                utility = self.minimax(openSpace, 'max', depth + 1)
                if minSeen == None:
                    minSeen = utility
                    move = openSpace
                elif utility < minSeen:
                    minSeen = utility
                    move = openSpace
            self.reverseMove(pos, player, conqueredPositions)
            return utility
        
    #UNDER EXPERIMENTATION
    def alphaBetaSearch(self, pos, player, depth, alpha, beta):
        maxDepth = 3
        conqueredPositions = self.makeMove(pos, player)
        if depth >= maxDepth or self.isComplete():
            self.reverseMove(pos, player, conqueredPositions)
            return self.utility()
        
        if player == 'max':
            maxSeen = None
            for openSpace in self.board.openPositions:
                utility = self.minimax(openSpace, 'min', depth + 1)
                if maxSeen == None:
                    maxSeen = utility
                    move = openSpace
                elif utility > maxSeen:
                    maxSeen = utility
                    move = openSpace
            self.reverseMove(pos, player, conqueredPositions)
            return utility
        else:
            minSeen = None
            for openSpace in self.board.openPositions:
                utility = self.minimax(openSpace, 'max', depth + 1)
                if minSeen == None:
                    minSeen = utility
                    move = openSpace
                elif utility < minSeen:
                    minSeen = utility
                    move = openSpace
            self.reverseMove(pos, player, conqueredPositions)
            return utility
        
    def makeMove(self, move, player):
        positionsConquered = []
        
        #remove move from list of open positions
        if move in self.board.openPositions:
            idx = self.board.openPositions.index(move)
            del self.board.openPositions[idx]
        else:
            print "Invalid Move"
            return
        
        surroundingMax = self.checkForSurroundingMax(move)
        surroundingMin = self.checkForSurroundingMin(move)
        
        if player == 'max':
            self.board.maxPlayerTaken.append(move)
            
            #Blitz: Conquer min players piece
            if len(surroundingMax) != 0:
                for pos in surroundingMin:
                    idx = self.board.minPlayerTaken.index(pos)
                    del self.board.minPlayerTaken[idx]
                    self.board.maxPlayerTaken.append(pos)
                    positionsConquered.append(pos)
                
        elif player == 'min':
            self.board.minPlayerTaken.append(move)
            
            #Blitz: Conquer max players piece
            if len(surroundingMin) != 0:
                for pos in surroundingMax:
                    idx = self.board.maxPlayerTaken.index(pos)
                    del self.board.maxPlayerTaken[idx]
                    self.board.minPlayerTaken.append(pos)
                    positionsConquered.append(pos)
                    
        return positionsConquered
                    
    def reverseMove(self, pos, player, positionsConquered):
        if player == 'max':
            idx = self.board.maxPlayerTaken.index(pos)
            del self.board.maxPlayerTaken[idx]
            self.board.openPositions.append(pos)
            for tile in positionsConquered:
                idx = self.board.maxPlayerTaken.index(tile)
                del self.board.maxPlayerTaken[idx]
                self.board.minPlayerTaken.append(tile)
                
        else:
            idx = self.board.minPlayerTaken.index(pos)
            del self.board.minPlayerTaken[idx]
            self.board.openPositions.append(pos)
            for tile in positionsConquered:
                idx = self.board.minPlayerTaken.index(tile)
                del self.board.minPlayerTaken[idx]
                self.board.maxPlayerTaken.append(tile)
                        
    def checkForSurroundingMax(self, pos):
        surroundingMax = []
        for neighbor in self.getNeighbors(pos):
            if neighbor in self.board.maxPlayerTaken:
                surroundingMax.append(neighbor)
        return surroundingMax
        
    def checkForSurroundingMin(self, pos):
        surroundingMin = []
        for neighbor in self.getNeighbors(pos):
            if neighbor in self.board.minPlayerTaken:
                surroundingMin.append(neighbor)
        return surroundingMin
            
    def getNeighbors(self, pos):
        neighbors = []
        if pos % 6 != 0:
            neighbors.append(pos-1)
        if pos / 6 != 0:
            neighbors.append(pos-6)
        if pos % 6 != 5:
            neighbors.append(pos+1)
        if pos / 6 != 5:
            neighbors.append(pos+6)
        return neighbors

20 16


In [103]:
newboard = board("game_boards/Keren.txt")
newWarGame = warGame(newboard)
newWarGame.runGame()
newWarGame.board.printBoard()
print "Max Score: ", newWarGame.maxScore 
print "Min Score: ", newWarGame.minScore

 
[A] [A] [B] [A] [A] [B]  
[B] [B] [B] [B] [B] [B]  
[A] [A] [B] [B] [B] [B]  
[A] [A] [A] [A] [B] [B]  
[A] [A] [A] [A] [A] [B]  
[A] [A] [A] [A] [A] [B]  
Max Score:  20
Min Score:  16


In [104]:
newboard = board("game_boards/Narvik.txt")
newWarGame = warGame(newboard)
newWarGame.runGame()
newWarGame.board.printBoard()
print "Max Score: ", newWarGame.maxScore 
print "Min Score: ", newWarGame.minScore

 
[A] [A] [B] [A] [A] [B]  
[A] [A] [A] [A] [B] [B]  
[A] [A] [B] [B] [B] [B]  
[A] [A] [A] [B] [B] [B]  
[B] [A] [A] [A] [A] [B]  
[B] [B] [A] [A] [A] [B]  
Max Score:  902
Min Score:  898


In [105]:
newboard = board("game_boards/Smolensk.txt")
newWarGame = warGame(newboard)
newWarGame.runGame()
newWarGame.board.printBoard()
print "Max Score: ", newWarGame.maxScore 
print "Min Score: ", newWarGame.minScore

 
[A] [B] [B] [A] [A] [A]  
[A] [B] [B] [A] [A] [B]  
[B] [B] [B] [A] [A] [B]  
[B] [B] [A] [A] [A] [A]  
[B] [A] [A] [A] [B] [A]  
[A] [A] [A] [A] [A] [B]  
Max Score:  879
Min Score:  774


In [106]:
newboard = board("game_boards/Westerplatte.txt")
newWarGame = warGame(newboard)
newWarGame.runGame()
newWarGame.board.printBoard()
print "Max Score: ", newWarGame.maxScore 
print "Min Score: ", newWarGame.minScore

 
[A] [A] [B] [B] [B] [A]  
[A] [B] [A] [B] [B] [B]  
[B] [A] [B] [B] [B] [B]  
[A] [A] [B] [B] [B] [B]  
[B] [A] [A] [B] [A] [B]  
[A] [B] [B] [B] [B] [B]  
Max Score:  28
Min Score:  44
